# Structured Outputs with OpenAI APIs

Reference blog: https://openai.com/index/introducing-structured-outputs-in-the-api/

Reference documentation: https://platform.openai.com/docs/guides/structured-outputs/introduction

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade python-dotenv

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
from google.colab import userdata

# client session
client = OpenAI(api_key=OPENAI_API_KEY)

## Simple Example of Structured Outputs

In [ ]:
# Pydantic object

class InsurancePolicy(BaseModel):
    policy_number: str
    policy_type: str  # e.g., 'health', 'life', 'auto'
    start_date: str
    end_date: str
    premium_amount: float
    coverage_details: str


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the insurance policy information."},
        {"role": "user", "content": "John Doe has a health insurance policy with policy number 123456789. The policy started on January 1, 2023, and ends on \
        December 31, 2023. The premium amount is $500. The coverage includes hospitalization and outpatient services."},
    ],
    response_format=InsurancePolicy,
)


event = completion.choices[0].message.parsed

In [ ]:
event

InsurancePolicy(policy_number='123456789', policy_type='Health Insurance', start_date='January 1, 2023', end_date='December 31, 2023', premium_amount=500.0, coverage_details='Hospitalization and outpatient services.')

## Structuring Chain-of-Thought Responses

In [ ]:
from pydantic import BaseModel

class Step(BaseModel):
    explanation: str
    output: str

class InsurancePolicyReasoning(BaseModel):
    steps: list[Step]
    final_answer: InsurancePolicy

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the insurance policy information step by step."},
        {"role": "user", "content": "During the call, John Doe mentioned that he has a health insurance policy with policy number 123456789. \
        The policy started on January 1, 2023, and ends on December 31, 2023. The premium amount is $500. The coverage includes hospitalization and outpatient services."},
    ],
    response_format=InsurancePolicyReasoning,
)

insurance_policy_reasoning = completion.choices[0].message.parsed


In [ ]:
insurance_policy_reasoning

InsurancePolicyReasoning(steps=[Step(explanation='Identify the policy number mentioned in the conversation.', output='The policy number is 123456789.'), Step(explanation='Determine the type of insurance policy based on the description provided.', output='The type of insurance policy is health insurance.'), Step(explanation='Extract the start date of the policy from the text.', output='The start date of the policy is January 1, 2023.'), Step(explanation='Extract the end date of the policy from the text.', output='The end date of the policy is December 31, 2023.'), Step(explanation='Determine the premium amount for the policy as mentioned in the conversation.', output='The premium amount is $500.'), Step(explanation='Extract the coverage details included in the policy.', output='The coverage details include hospitalization and outpatient services.')], final_answer=InsurancePolicy(policy_number='123456789', policy_type='Health Insurance', start_date='January 1, 2023', end_date='December 3

## Structured Output with Function Calling

Useful when you are connecting the model to tools, functions, data, etc. in your system.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_policy_details",
            "description": "Get the details of a customer's insurance policy. Call this whenever you need to know the policy details, for example when a customer asks about their insurance coverage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "policy_number": {
                        "type": "string",
                        "description": "The customer's insurance policy number.",
                    },
                },
                "required": ["policy_number"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    },
    {
        "type": "function",
        "function": {
            "name": "verify_customer_identity",
            "description": "Verify the identity of a customer using various authentication mechanisms.",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "string",
                        "description": "The customer's unique ID.",
                    },
                    "authentication_method": {
                        "type": "string",
                        "description": "The method used for authentication (e.g., 'password', 'OTP', 'security_question').",
                    },
                },
                "required": ["customer_id", "authentication_method"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    }
]


In [ ]:
messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, I need information about my insurance policy. My policy number is 987654321."})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)


In [ ]:
response

ChatCompletion(id='chatcmpl-9tWOXis89lXSkoOmTl8D6eheUFBXs', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3D9GqHeXFxnSePsKy9VTkX4P', function=Function(arguments='{"policy_number":"987654321"}', name='get_policy_details'), type='function')]))], created=1723019593, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_845eaabc1f', usage=CompletionUsage(completion_tokens=18, prompt_tokens=170, total_tokens=188))

In [ ]:
messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. First, verify the customer's identity using the provided authentication method before retrieving any sensitive information."})
messages.append({"role": "user", "content": "Hi, I need information about my insurance policy. My customer ID is CUST12345 and I can verify my identity using my password."})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)


In [ ]:
response

ChatCompletion(id='chatcmpl-9tWUIzc27nNtW3beQAGil722OrjuS', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_JuzMkjypuFrQ3FdnFEa8jgx9', function=Function(arguments='{"customer_id":"CUST12345","authentication_method":"password"}', name='verify_customer_identity'), type='function')]))], created=1723019950, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_845eaabc1f', usage=CompletionUsage(completion_tokens=24, prompt_tokens=196, total_tokens=220))

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)

In [ ]:
response

ChatCompletion(id='chatcmpl-9tW5K0uhKdxj2UjSqxcmZXtyFrSTb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can help with that. Could you please provide your order ID so I can check the delivery date for you?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723018402, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_845eaabc1f', usage=CompletionUsage(completion_tokens=24, prompt_tokens=107, total_tokens=131))

## Structured Output with `response_format`

Useful when the model needs to respond to the user in a specified structured way.

In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "insurance_policy_response",
        "schema": {
            "type": "object",
            "properties": {
                "steps": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "explanation": {"type": "string"},
                            "output": {"type": "string"}
                        },
                        "required": ["explanation", "output"],
                        "additionalProperties": False
                    }
                },
                "final_answer": {
                    "type": "object",
                    "properties": {
                        "policy_number": {"type": "string"},
                        "policy_type": {"type": "string"},
                        "start_date": {"type": "string"},
                        "end_date": {"type": "string"},
                        "premium_amount": {"type": "number"},
                        "coverage_details": {"type": "string"}
                    },
                    "required": ["policy_number", "policy_type", "start_date", "end_date", "premium_amount", "coverage_details"],
                    "additionalProperties": False
                }
            },
            "required": ["steps", "final_answer"],
            "additionalProperties": False
        },
        "strict": True
    }
}


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Guide the user through extracting insurance policy details step by step."},
    {"role": "user", "content": "Hi, I need information about my insurance policy. My policy number is 987654321. The policy started on January 1, 2023, and ends on December 31, 2023. The premium amount is $500. The coverage includes hospitalization and outpatient services."}
]


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=messages,
    response_format=response_format
)

print(response.choices[0].message.content)


{"steps":[{"explanation":"Start by confirming the provided policy number to ensure it's in a valid format and corresponds to your records.","output":"Confirmed policy number: 987654321."},{"explanation":"Verify the start and end dates of the policy to ensure they are accurate and fall within the active period.","output":"Policy start date: January 1, 2023; Policy end date: December 31, 2023."},{"explanation":"Check the premium amount to ensure it matches your records and agree to the expected premium amount for the policy period.","output":"Premium amount is confirmed as $500."},{"explanation":"Review the coverage details to confirm they include what is outlined in your policy, such as hospitalization and outpatient services.","output":"Coverage includes: hospitalization and outpatient services."}],"final_answer":{"policy_number":"987654321","policy_type":"Health Insurance","start_date":"January 1, 2023","end_date":"December 31, 2023","premium_amount":500,"coverage_details":"Hospitaliz